In [ ]:
import numpy as np
import pandas as pd 
import altair as alt
import re 
from commons import data_processing
import matplotlib.pyplot as plt
from venn import venn
from matplotlib_venn import venn2

alt.data_transformers.disable_max_rows()

In [ ]:
pgc = data_processing.get_files(r'D:\DIA-NN\20221109\PGC', exts=['PGC.tsv'])[0]
rplc = data_processing.get_files(r'D:\DIA-NN\20221109\RPLC', exts=['RPLC.tsv'])[0]


In [ ]:
files = [rplc, pgc]
columns = ['RPLC', 'PGC']

df = pd.DataFrame()

for (col, file) in zip(columns, files):
    print(col, file)
    _df = pd.read_csv(file, delimiter='\t')
    _df.loc[:, 'column'] = col 
    _df.loc[:, 'sample'] = _df.Run.map(lambda x: x.split('_')[-3])
    _df.loc[:, 'tech_rep'] = _df.Run.map(lambda x: x.split('_')[-1])

    df = pd.concat([df, _df])
    df.reset_index(inplace=True, drop=True)
df

In [ ]:
d = df[(df["Proteotypic"]==1)&(df["Protein.Q.Value"]<=0.01)]
d = d.groupby(['sample', 'column', 'Protein.Ids']).mean()

samples = df['sample'].unique()

fig, axs = plt.subplots(1, 2)
i = 0
for s in samples:
    overlap = {}
    for c in columns:
        small = d.loc[(s, c), :]
        small.reset_index(inplace=True)
        overlap[c] = set(small["Protein.Ids"].tolist())
    venn(overlap, ax=axs[i])
    axs[i].set_title(s)

    i += 1

In [ ]:
alt.Chart(d.reset_index()).mark_bar().encode(
    x=alt.X('RT:Q', title='RT',
        bin=alt.Bin(
            step=5
        )),
    y=alt.Y('count():Q', title=''),
    color=alt.Color('column:N'),
    row='sample:N'
)